In [13]:
import pandas as pd
import numpy as np
# Load in the Google Analytics Sample Dataset
import bq_helper
from bq_helper import BigQueryHelper
google_analytics = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="data:google_analytics_sample")

Using Kaggle's public dataset BigQuery integration.


In [14]:
bq_assistant = BigQueryHelper("bigquery-public-data", "google_analytics_sample")
bq_assistant.list_tables()

Using Kaggle's public dataset BigQuery integration.


['ga_sessions_20160801',
 'ga_sessions_20160802',
 'ga_sessions_20160803',
 'ga_sessions_20160804',
 'ga_sessions_20160805',
 'ga_sessions_20160806',
 'ga_sessions_20160807',
 'ga_sessions_20160808',
 'ga_sessions_20160809',
 'ga_sessions_20160810',
 'ga_sessions_20160811',
 'ga_sessions_20160812',
 'ga_sessions_20160813',
 'ga_sessions_20160814',
 'ga_sessions_20160815',
 'ga_sessions_20160816',
 'ga_sessions_20160817',
 'ga_sessions_20160818',
 'ga_sessions_20160819',
 'ga_sessions_20160820',
 'ga_sessions_20160821',
 'ga_sessions_20160822',
 'ga_sessions_20160823',
 'ga_sessions_20160824',
 'ga_sessions_20160825',
 'ga_sessions_20160826',
 'ga_sessions_20160827',
 'ga_sessions_20160828',
 'ga_sessions_20160829',
 'ga_sessions_20160830',
 'ga_sessions_20160831',
 'ga_sessions_20160901',
 'ga_sessions_20160902',
 'ga_sessions_20160903',
 'ga_sessions_20160904',
 'ga_sessions_20160905',
 'ga_sessions_20160906',
 'ga_sessions_20160907',
 'ga_sessions_20160908',
 'ga_sessions_20160909',


In [15]:
# Print out the first few rows of a session to see the dataframe
bq_assistant.head("ga_sessions_20160801", num_rows=3)

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,channelGrouping,socialEngagementType
0,None,1,1470117657,1470117657,20160801,"{'visits': 1, 'hits': 3, 'pageviews': 3, 'time...","{'referralPath': '/yt/about/', 'campaign': '(n...","{'browser': 'Internet Explorer', 'browserVersi...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 23, 'minu...",7194065619159478122,None,Social,Not Socially Engaged
1,None,151,1470083489,1470083489,20160801,"{'visits': 1, 'hits': 3, 'pageviews': 3, 'time...","{'referralPath': '/yt/about/', 'campaign': '(n...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 13, 'minu...",8159312408158297118,None,Social,Not Socially Engaged
2,None,1,1470052694,1470052694,20160801,"{'visits': 1, 'hits': 4, 'pageviews': 3, 'time...","{'referralPath': '/yt/about/', 'campaign': '(n...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Asia', 'subContinent': 'Southea...",[],"[{'hitNumber': 1, 'time': 0, 'hour': 4, 'minut...",9236304747882138291,None,Social,Not Socially Engaged


In [16]:
# Printing out the schema
bq_assistant.table_schema("ga_sessions_20160801")


/src/bq-helper/bq_helper.py:63: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  'fields': pd.np.nan,


,name,type,mode,description
0,visitorId,INTEGER,NULLABLE,None
1,visitNumber,INTEGER,NULLABLE,None
2,visitId,INTEGER,NULLABLE,None
3,visitStartTime,INTEGER,NULLABLE,None
4,date,STRING,NULLABLE,None
...,...,...,...,...
331,hits.publisher_infos.dfpNetworkId,STRING,NULLABLE,None
332,fullVisitorId,STRING,NULLABLE,None
333,userId,STRING,NULLABLE,None
334,channelGrouping,STRING,NULLABLE,None


What is the total number of transactions generated per device browser in July 2017?

In [17]:
query1 = """SELECT
device.browser,
SUM ( totals.transactions ) AS total_transactions
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
_TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
GROUP BY
device.browser
ORDER BY
total_transactions DESC;
        """
response1 = google_analytics.query_to_pandas_safe(query1)
response1.head(10)

,browser,total_transactions
0,Chrome,984.0
1,Safari,65.0
2,Firefox,12.0
3,Internet Explorer,7.0
4,Opera,2.0
5,Edge,2.0
6,Opera Mini,NaN
7,Safari (in-app),NaN
8,UC Browser,NaN
9,YaBrowser,NaN


The real bounce rate is defined as the percentage of visits with a single pageview. What was the real bounce rate per traffic source?

In [18]:
query2 = """SELECT
source,
total_visits,
total_no_of_bounces,
( ( total_no_of_bounces / total_visits ) * 100 ) AS bounce_rate
FROM (
SELECT
trafficSource.source AS source,
COUNT ( trafficSource.source ) AS total_visits,
SUM ( totals.bounces ) AS total_no_of_bounces
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
_TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
GROUP BY
source )
ORDER BY
total_visits DESC;
        """
response2 = google_analytics.query_to_pandas_safe(query2)
response2.head(10)

,source,total_visits,total_no_of_bounces,bounce_rate
0,google,38400,19798.0,51.557292
1,(direct),19891,8606.0,43.265799
2,youtube.com,6351,4238.0,66.729649
3,analytics.google.com,1972,1064.0,53.955375
4,Partners,1788,936.0,52.348993
5,m.facebook.com,669,430.0,64.275037
6,google.com,368,183.0,49.728261
7,dfa,302,124.0,41.059603
8,sites.google.com,230,97.0,42.173913
9,facebook.com,191,102.0,53.403141


What is the average amount of money spent per session in July 2017?

In [19]:
query6 = """SELECT
( SUM(total_transactionrevenue_per_user) / SUM(total_visits_per_user) ) AS
avg_revenue_by_user_per_visit
FROM (
SELECT
fullVisitorId,
SUM( totals.visits ) AS total_visits_per_user,
SUM( totals.transactionRevenue ) AS total_transactionrevenue_per_user
FROM
`bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
_TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
AND
totals.visits > 0
AND totals.transactions >= 1
AND totals.transactionRevenue IS NOT NULL
GROUP BY
fullVisitorId );
        """
response6 = google_analytics.query_to_pandas_safe(query6, max_gb_scanned=10)
response6.head(10)

,avg_revenue_by_user_per_visit
0,1.207558e+08


What is the sequence of pages viewed?

In [21]:
query7 = """SELECT
fullVisitorId,
visitId,
visitNumber,
hits.hitNumber AS hitNumber,
hits.page.pagePath AS pagePath
FROM
`bigquery-public-data.google_analytics_sample.ga_sessions_*`,
UNNEST(hits) as hits
WHERE
_TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
AND
hits.type="PAGE"
ORDER BY
fullVisitorId,
visitId,
visitNumber,
hitNumber;
        """
response7 = google_analytics.query_to_pandas_safe(query7, max_gb_scanned=10)
response7.head(10)

,fullVisitorId,visitId,visitNumber,hitNumber,pagePath
0,0000062267706107999,1499645960,1,1,/home
1,0000085059828173212,1500505105,1,1,/asearch.html
2,0000436683523507380,1500504900,1,1,/home
3,0000436683523507380,1500504900,1,2,/google+redesign/electronics/power/clip+compac...
4,0000436683523507380,1500989127,2,1,/home
5,0000436683523507380,1500989127,2,2,/google+redesign/bags/backpacks/waterproof+bac...
6,0000436683523507380,1500989127,2,3,/home
7,0000436683523507380,1500989127,2,4,/home
8,0000436683523507380,1500989127,2,5,/google+redesign/electronics
9,0000436683523507380,1500989127,2,6,/google+redesign/office/notebooks+journals/goo...


Small examples of trends from google analytics. We could try to implement these algorithms on our own too.